In [ ]:
import numpy as np
from plotly import graph_objects as go, io as pio
from plotly.subplots import make_subplots
from easy_o3d import utils

In [ ]:
source_path = "test_data/suzanne.ply"
source = utils.eval_data(data=source_path, number_of_points=10000)

points = np.asarray(source.points)
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]

suzanne = go.Scatter3d(x=x,
                       y=y,
                       z=z,
                       mode="markers",
                       marker=dict(size=1, color=points[:, 1], colorscale="blues"),
                       hoverinfo="name",
                       name="Suzanne")

In [ ]:
scene_id = 20
path_to_scene_camera_json = "test_data/bop_data/obj_of_interest/train_pbr/000000/scene_camera.json"
path_to_camera_json = "test_data/bop_data/obj_of_interest/camera.json"
camera_parameters = utils.get_camera_parameters_from_blenderproc_bopwriter(path_to_scene_camera_json,
                                                                           path_to_camera_json,
                                                                           scene_id)[0]

color = f"test_data/bop_data/obj_of_interest/train_pbr/000000/rgb/{str(scene_id).zfill(6)}.png"
depth = f"test_data/bop_data/obj_of_interest/train_pbr/000000/depth/{str(scene_id).zfill(6)}.png"

target = utils.eval_data(data=[color, depth],
                         camera_intrinsic=camera_parameters.intrinsic,
                         camera_extrinsic=camera_parameters.extrinsic,
                         depth_scale=1000.0,
                         depth_trunc=2.0)

target = utils.process_point_cloud(point_cloud=target,
                                   remove_outlier=utils.OutlierTypes.STATISTICAL,
                                   outlier_std_ratio=10.0)

In [ ]:
points = np.asarray(target.points)
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]
color = np.asarray(target.colors)

suzanne_on_chair = go.Scatter3d(x=x,
                                y=y,
                                z=z,
                                mode="markers",
                                marker=dict(size=1, color=color),
                                hoverinfo="name",
                                name="Suzanne on chair")

In [ ]:
fig = make_subplots(rows=1,
                    cols=2,
                    column_widths=[0.5, 0.5],
                    horizontal_spacing=0,
                    vertical_spacing=0,
                    specs=[[dict(type='Scatter3d'), dict(type='Scatter3d')]])

fig.add_trace(suzanne, row=1, col=1)
fig.add_trace(suzanne_on_chair, row=1, col=2)

fig.update_layout(scene1=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  scene2=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=500,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_dragmode='orbit',
                  showlegend=False)

In [ ]:
pio.write_html(fig,
               file="test_data/test_data.html",
               full_html=False,
               include_plotlyjs='cdn')

In [ ]:
gt_pose = utils.get_ground_truth_pose_from_file("test_data/ground_truth_pose.json")
source.transform(gt_pose)

points = np.asarray(source.points)
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]

suzanne = go.Scatter3d(x=x,
                       y=y,
                       z=z,
                       mode="markers",
                       marker=dict(size=1, color=points[:, 1], colorscale="reds"),
                       hoverinfo="name",
                       name="Suzanne")

points = np.asarray(target.points)
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]

suzanne_on_chair = go.Scatter3d(x=x,
                                y=y,
                                z=z,
                                mode="markers",
                                marker=dict(size=1, color=points[:, 1], colorscale='greys'),
                                hoverinfo="name",
                                name="Suzanne on chair")

fig = go.Figure([suzanne, suzanne_on_chair])
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=500,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_dragmode="orbit",
                  showlegend=False)

In [ ]:
pio.write_html(fig,
               file="test_data/registration_result.html",
               full_html=False,
               include_plotlyjs='cdn')